### Script Overview:
This script performs the following tasks:
1. Loads an image using OpenCV.
2. Divides the image into a grid of tiles based on specified rows and columns.
3. Displays the original image with grid lines overlayed.
4. Extracts and displays each tile individually.
5. Arranges and displays all tiles in a single grid layout.
6. Encodes each tile image to base64.
7. Sends each encoded tile to the OpenAI API to identify any machine components.
8. Prints out the response from the OpenAI API for each tile.

### Import Libraries 

In [ ]:
# Import necessary libraries
import os  # For environment variables and file path manipulation
import io  # For in-memory byte streams
import base64  # For encoding images to base64 strings
import cv2  # For image loading and processing
import matplotlib.pyplot as plt  # For displaying images
import openai  # For interacting with the OpenAI API
from PIL import Image  # For image processing
from dotenv import load_dotenv  # For loading environment variables from a .env file
import requests  # For making HTTP requests

### Load Environment Variables

In [ ]:
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

### Load and Edit Image

In [ ]:
# Load the image using OpenCV
image_path = 'conveyor_machine.png'  # Specify the path to your image here
# image_path = 'packaging_machine.png'  # Specify the path to your image here
image = cv2.imread(image_path)  # Read the image from the specified path

# Check if the image was loaded successfully
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

# Convert the image from BGR (OpenCV default) to RGB for displaying with matplotlib
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define the number of rows and columns for the grid (e.g., 3x3 grid)
num_rows = 3  # Number of rows in the grid
num_cols = 3  # Number of columns in the grid

# Calculate the height and width of each tile
tile_height = image.shape[0] // num_rows  # Height of each tile
tile_width = image.shape[1] // num_cols  # Width of each tile

# Display the original image with grid lines
plt.figure(figsize=(8, 8))  # Create a figure with a specified size
plt.imshow(image_rgb)  # Display the image
plt.title("Original Image with Grid Lines")  # Set the title of the plot

# Draw horizontal grid lines
for row in range(1, num_rows):
    y = row * tile_height  # Calculate the y-coordinate for the horizontal line
    plt.axhline(y=y, color='red', linestyle='--', linewidth=1)  # Draw the horizontal line

# Draw vertical grid lines
for col in range(1, num_cols):
    x = col * tile_width  # Calculate the x-coordinate for the vertical line
    plt.axvline(x=x, color='red', linestyle='--', linewidth=1)  # Draw the vertical line

# Remove axis ticks and labels for a cleaner look
plt.axis("off")  # Hide the axes
plt.show()  # Display the plot

# Initialize a list to store the tiles
tiles = []  # List to hold each tile extracted from the image

# Loop through rows and columns to extract each tile
for row in range(num_rows):
    for col in range(num_cols):
        # Calculate the starting and ending pixel coordinates for each tile
        y_start = row * tile_height  # Starting y-coordinate
        y_end = y_start + tile_height  # Ending y-coordinate
        x_start = col * tile_width  # Starting x-coordinate
        x_end = x_start + tile_width  # Ending x-coordinate

        # Extract the tile from the original image
        tile = image_rgb[y_start:y_end, x_start:x_end]  # Slice the image array to get the tile

        # Append the tile to the list
        tiles.append(tile)  # Add the tile to the tiles list

        # Display each tile individually
        plt.figure(figsize=(3, 3))  # Create a figure for the tile
        plt.title(f"Tile {row * num_cols + col + 1}")  # Set the title with the tile number
        plt.imshow(tile)  # Display the tile image
        plt.axis("off")  # Hide the axes
        plt.show()  # Display the plot

# Plot all tiles in a single figure, arranged in a grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 10))  # Create subplots

# Flatten the axes array for easy iteration
axes = axes.flatten()  # Flatten the 2D array of axes into a 1D array

# Loop through each tile and corresponding axis to display them
for idx, (tile, ax) in enumerate(zip(tiles, axes)):
    # Display each tile in the respective subplot
    ax.imshow(tile)  # Show the tile image on the subplot
    ax.set_title(f"Tile {idx + 1}")  # Set the title with the tile number
    ax.axis("off")  # Hide the axes for a cleaner look

# Adjust layout to prevent overlap
plt.tight_layout()  # Automatically adjust subplot parameters for a neat layout
plt.show()  # Display the grid of tiles

### Function to encode the image to base64

In [ ]:
def encode_image_to_base64(image):
    """
    Encodes a PIL Image object to a base64 string.

    Args:
        image (PIL.Image.Image): The image to encode.

    Returns:
        str: The base64 encoded string of the image.
    """
    buffered = io.BytesIO()  # Create an in-memory bytes buffer
    image.save(buffered, format="PNG")  # Save the image to the buffer in PNG format
    return base64.b64encode(buffered.getvalue()).decode("utf-8")  # Encode and return as string


### Function to send a request with the encoded image and prompt

In [ ]:
def identify_components_in_tile(tile_image):
    """
    Sends a tile image to the OpenAI API to identify any machine components.

    Args:
        tile_image (PIL.Image.Image): The tile image to analyze.
    """
    # Encode the tile image to base64
    base64_image = encode_image_to_base64(tile_image)  # Convert image to base64 string

    # Define headers and payload for the API request
    headers = {
        "Content-Type": "application/json",  # Specify the content type
        "Authorization": f"Bearer {api_key}"  # Include the API key in the header
    }

    # Define the prompt and payload for the request
    payload = {
        "model": "gpt-4o",  # Specify the model to use (Note: 'gpt-4o' may need to be adjusted)
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Please identify any machine components in this image."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"  # Include the image in data URL format
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300  # Set the maximum number of tokens in the response
    }

    # Send the request to the OpenAI API
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json=payload
    )

    # Output the response content
    if response.status_code == 200:
        # Extract the response message content
        response_message = response.json()["choices"][0]["message"]["content"]
        print("Detected components:", response_message)  # Print the detected components
    else:
        # Print error details if the request failed
        print("Error:", response.status_code, response.text)



### Iterate over each tile for analysis

In [ ]:

for index, tile in enumerate(tiles):
    print(f"\nAnalyzing Tile {index + 1}")  # Indicate which tile is being analyzed
    tile_image = Image.fromarray(tile)  # Convert the NumPy array to a PIL Image
    identify_components_in_tile(tile_image)  # Call the function to analyze the tile